Reading in windows in the 7 cluster dataset based on genetic distances.

In [1]:
%run ../scripts/notebook_settings_lean.py
#from horizonplot import horizonplot

/faststorage/project/baboondiversity/people/eriks/second_analysis_baboons/scripts/notebook_settings_lean.py:7: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina', 'png')


In [2]:
#Functions to generate the stacked area plot df

def length_to_windows_cm(length_df, window_size):
    pops = sorted(length_df["pop"].unique())
    window_l = list(np.arange(length_df.M.values[0], length_df.M.values[-1], window_size))
    overall_dict = {"windows": window_l}
    for p in pops:
        overall_dict[p] = np.zeros(len(window_l))
    for i in range(len(window_l)):
        w = window_l[i]
        # Finding all paintings that start in this window.
        s_df = length_df.loc[(length_df.M >= w) & (length_df.M < w+window_size)]
        for index, row in s_df.iterrows():
            # x decides how many windows it should go beyond the current.
            # if x==0, it both starts and ends in the same window
            x = int((row.M-w+row.M_length)//window_size)
            if x == 0:
                overall_dict[row["pop"]][i] += float(row.M_length)
            # Otherwise, the current window is filled from row.cM to window end
            else:
                overall_dict[row["pop"]][i] += float((w+window_size)-row.M)
                # Then iterated for each full window (which is all windows < x)
                for s in range(x-1):
                    overall_dict[row["pop"]][i+1+s] += float(window_size)
                # Then finally added the ending window
                # print(row.paint_length+row.pos, (w+x*window_size), x, w)
                overall_dict[row["pop"]][i+x] += float(row.M_length+row.M-(w+x*window_size))
    df = pd.DataFrame(overall_dict)
    return df

def generate_spot_df(single_p_df, map_dict):
    single_p_df["pop"] = single_p_df["paint"].map(map_dict)
    shift_spots = (single_p_df["pop"] != single_p_df["pop"].shift())
    #df based on this subset
    spot_df = single_p_df.loc[shift_spots].copy()
    #Finding the start of the next painting, by removing the first pos and adding the last snp.
    #And then calculating the difference.
    spot_df["paint_length"] = (np.append(spot_df["pos"][1:].values, single_p_df.iloc[-1]["pos"])-spot_df["pos"].values)
    spot_df["end_pos"] = (np.append(spot_df["pos"][1:].values, single_p_df.iloc[-1]["pos"])-1)
    return spot_df

def create_paint_df_lengths(paintfile, idfile):
    #various supplemental dataframes/lists
    end_pos = pos_list.values[-1]
    
    map_dict = {}
    for i, row in idfile.iterrows():
        map_dict[row.ID_index*2+1] = row["pop"]
        map_dict[row.ID_index*2+2] = row["pop"]

    df_l = []
    f = open(paintfile, "r")
    ad_lines = f.readlines()
    ind = ""
    hap = ""
    for p_num in range(1, len(ad_lines)):
        l = ad_lines[p_num].strip().split(" ")
        if l[0] == "HAP":
            ind = l[2]
            if hap == 1:
                hap = 2
            else:
                hap = 1
        else:
            paint_line = l[1:]
            single_p_df = pd.DataFrame(data={"pos": pos_list, "paint": paint_line})
            single_p_df = single_p_df.astype(int)
            spot_df = generate_spot_df(single_p_df, map_dict)
            spot_df["individual"] = ind
            spot_df["haplotype"] = hap
            df_l.append(spot_df)
    all_df = pd.concat(df_l)
    return all_df

In [3]:
idfile_path = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_7_cluster_females.ids"
idfile = pd.read_csv(idfile_path, sep=" ", names=["PGDP_ID", "pop", "inclusion"])
idfile["ID_index"] = idfile.index
recomb_map_path = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chr{}/chr{}.females.recombfile"

os.makedirs("../steps/paint_summary/", exist_ok=True)
chrom_list = [] #["8", "X"] # ["X"]
window_length = 100000
df_l = []
for chrom in chrom_list:
    recomb_map = pd.read_csv(recomb_map_path.format(chrom, chrom), sep=" ")
    pos_list = recomb_map["start.pos"]
    for population in ["cyno_anubis_t"]:
        print(population)
        pop_path = "/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/" \
                        "chr{}_7_cluster_large/{}_chr{}_7_cluster_large/chr{}.samples.out"\
                        .format(chrom, population, chrom, chrom)
        length_df = create_paint_df_lengths(pop_path, idfile)
        length_df["chrom"] = chrom
        length_df["C_origin"] = population
        df_l.append(length_df)

In [4]:
#df_lengths_7 = pd.concat(df_l)
#df_lengths_7.to_csv("../steps/paint_summary/"+"paint_lengths_7_cluster.txt", sep=" ", index=False)
df_lengths_7 = pd.read_csv("../steps/paint_summary/"+"paint_lengths_7_cluster.txt", sep=" ")

/scratch/3298076/ipykernel_1564/685277103.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lengths_7 = pd.read_csv("../steps/paint_summary/"+"paint_lengths_7_cluster.txt", sep=" ")


In [5]:
df_lengths_7.chrom = df_lengths_7.chrom.astype(str)

For each chrom, I will map the genetic distance to it found in the recomb_map, and then create windows based on the genetic distance.

In [6]:
recomb_map_path.format("8", "8")

'/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chr8/chr8.females.recombfile'

In [7]:
recomb_map_path = "/home/eriks/baboondiversity/data/PG_panu3_recombination_map/genetic_map_chr{}.txt"

os.makedirs("../steps/paint_summary/", exist_ok=True)
chrom_list = ["X"] # ["X"]
window_length_m = 10

for chrom in chrom_list:
    recomb_map = pd.read_csv(recomb_map_path.format(chrom), sep=" ")
    recomb_map["M"] = recomb_map["Genetic_Map(cM)"]*100
    cm_mapping = dict(zip(recomb_map["position"], recomb_map["M"]))
    for population in ["cyno_anubis_t"]:
        length_df = df_lengths_7.loc[df_lengths_7.chrom == chrom]
        length_df["M"] = length_df.pos.map(cm_mapping)
        for ind in idfile.loc[(idfile["pop"] == population) & (idfile.inclusion == 1)].PGDP_ID[:1]:
            print(ind)
            name = population+"_chr"+chrom+"_7_cluster_"+ind+"_cm.txt"
            ind_df = length_df.loc[(length_df.individual==ind)]
            ind_df["M_length"] = ind_df.shift(periods=-1).M-ind_df.M
            ind_df.loc[ind_df.M_length <= 0, "M_length"] = 0 # The last positions per sampling
            ind_df["M_end"] = ind_df.M+ind_df.M_length
            ind_df = ind_df.dropna() #Is only the last interval in the whole dataframe
            window_df = length_to_windows_cm(ind_df, window_size=window_length_m)
            #window_df.to_csv("../steps/paint_summary/"+name, sep=" ", index=False)

/scratch/3298076/ipykernel_1564/3182861604.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  length_df["M"] = length_df.pos.map(cm_mapping)


Sci_16066


/scratch/3298076/ipykernel_1564/3182861604.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind_df["M_length"] = ind_df.shift(periods=-1).M-ind_df.M
/scratch/3298076/ipykernel_1564/3182861604.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind_df["M_end"] = ind_df.M+ind_df.M_length


In [14]:
length_df

,pos,paint,pop,paint_length,end_pos,individual,haplotype,chrom,C_origin,M
18305990,2500022,49,cyno_western,18543,2518564,Sci_16066,1,X,cyno_anubis_t,157.400117
18305991,2518565,138,ursinus,7119,2525683,Sci_16066,1,X,cyno_anubis_t,157.401418
18305992,2525684,49,cyno_western,1989,2527672,Sci_16066,1,X,cyno_anubis_t,157.401952
18305993,2527673,142,hamadryas,11289,2538961,Sci_16066,1,X,cyno_anubis_t,157.886163
18305994,2538962,172,kindae,14515,2553476,Sci_16066,1,X,cyno_anubis_t,158.375758
...,...,...,...,...,...,...,...,...,...,...
26295354,143613009,186,kindae,30898,143643906,PD_0774,2,X,cyno_anubis_t,8590.900776
26295355,143643907,112,cyno_western,23824,143667730,PD_0774,2,X,cyno_anubis_t,8591.643177
26295356,143667731,138,ursinus,492,143668222,PD_0774,2,X,cyno_anubis_t,8594.922532
26295357,143668223,114,cyno_western,11523,143679745,PD_0774,2,X,cyno_anubis_t,8594.932964


In [8]:
ind_df

,pos,paint,pop,paint_length,end_pos,individual,haplotype,chrom,C_origin,M,M_length,M_end
18305990,2500022,49,cyno_western,18543,2518564,Sci_16066,1,X,cyno_anubis_t,157.400117,0.001301,157.401418
18305991,2518565,138,ursinus,7119,2525683,Sci_16066,1,X,cyno_anubis_t,157.401418,0.000534,157.401952
18305992,2525684,49,cyno_western,1989,2527672,Sci_16066,1,X,cyno_anubis_t,157.401952,0.484211,157.886163
18305993,2527673,142,hamadryas,11289,2538961,Sci_16066,1,X,cyno_anubis_t,157.886163,0.489595,158.375758
18305994,2538962,172,kindae,14515,2553476,Sci_16066,1,X,cyno_anubis_t,158.375758,0.002177,158.377935
...,...,...,...,...,...,...,...,...,...,...,...,...
18497183,143675429,75,papio,170,143675598,Sci_16066,2,X,cyno_anubis_t,8596.772631,0.007408,8596.780038
18497184,143675599,146,hamadryas,1461,143677059,Sci_16066,2,X,cyno_anubis_t,8596.780038,0.510220,8597.290258
18497185,143677060,188,kindae,2211,143679270,Sci_16066,2,X,cyno_anubis_t,8597.290258,0.432871,8597.723129
18497186,143679271,76,papio,483,143679753,Sci_16066,2,X,cyno_anubis_t,8597.723129,0.190371,8597.913499


In [9]:
ind_df.M_length.sum()/((recomb_map.M.max()-ind_df.M.min())*20)
# Estimate of lost M

0.9998094424999993

In [10]:
ind_df.M_length.sum(), ind_df.M_end.sum()-ind_df.M.sum() # No observed floating point imprecision

(168832.5961588611, 168832.5961587429)

In [11]:
window_df

,windows,anubis_ethiopia,cyno_western,hamadryas,kindae,papio,ursinus
0,157.400117,36.003927,42.125697,46.700942,43.812442,9.230299,22.126693
1,167.400117,19.947162,16.171685,14.883380,66.216799,1.503495,81.277480
2,177.400117,13.926388,46.783631,20.419971,83.092638,5.720774,30.056598
3,187.400117,37.119735,45.604482,9.308489,61.518264,7.522332,38.926698
4,197.400117,39.213361,22.731334,20.577493,112.290453,5.187359,0.000000
...,...,...,...,...,...,...,...
840,8557.400117,3.181486,110.137681,12.364485,2.724882,4.540595,67.050871
841,8567.400117,3.668018,141.266187,14.080080,11.099117,3.860597,26.026001
842,8577.400117,1.587607,135.943159,4.505965,50.960476,2.218399,4.784394
843,8587.400117,22.452847,56.691575,17.475419,97.484371,3.562413,2.333375


In [12]:
window_df.iloc[:,1:7].sum(axis=1)

0      200.000000
1      200.000000
2      200.000000
3      200.000000
4      200.000000
          ...    
840    200.000000
841    200.000000
842    200.000000
843    200.000000
844     32.596159
Length: 845, dtype: float64

In [13]:
window_df.iloc[:,1:7].sum(axis=1).sum() # There is some data which goes missing. Determine which case leads to this.
# It happens in the first window, so it cannot be only in the range

168832.59615886112